In [1]:
import pandas as pd
import json
import os
from datetime import date

# Collecting the data from Twitter

In [2]:
# Import the Twython class
from twython import Twython
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])


In [3]:
import pysad

In [4]:
import importlib
importlib.reload(pysad)

<module 'pysad' from '/home/benjamin/Documents/EPFL/Research/sad/sad_tweets/pysad.py'>

In [5]:
init_accounts = pysad.initial_accounts()

In [6]:
init_accounts.list()

['swiss_climate_regular',
 'swiss_climate_controversial',
 'swiss_immigration',
 'french_tech_lesechos']

In [40]:

    
######Choose a category##############    
category_name = 'swiss_climate_controversial'
#category_name = 'swiss_climate_regular'
#category_name = 'french_tech_lesechos'
#category_name = 'swiss_immigration'
#####################################

username_list = init_accounts.accounts(category_name)

# create the path to save the experiment indexed with a date
today = date.today()
date_string = today.strftime("%Y%m%d")
print("date string =", date_string)

#date_string = '191128'

tweet_data_path_list = ['tweetdata', category_name, date_string]
results_data_path_list = ['resultsdata', category_name, date_string]
#get_tweets = python_tweets.get_user_timeline(screen_name = username,  
#         count = 200, include_rts = True)

def initialize_folder(path_folder_list):
    folder_concat = ''
    for folder in path_folder_list[:-1]:
        folder_concat += folder + '/'
        if not os.path.isdir(folder_concat):
            os.mkdir(folder_concat)
            print('Path created:',folder_concat)
    # Special treatment for the last folder
    folder_concat += path_folder_list[-1] + '/'
    if not os.path.isdir(folder_concat):
        os.mkdir(folder_concat)
        print('Path created:',folder_concat)
    else:
        for f in os.listdir(folder_concat):
            os.remove(os.path.join(folder_concat, f))
        print('Cleaned path',folder_concat)
    return folder_concat

tweet_data_path = initialize_folder(tweet_data_path_list)
results_data_path = initialize_folder(results_data_path_list)

date string = 20200131
Path created: tweetdata/swiss_climate_controversial/20200131/
Path created: resultsdata/swiss_climate_controversial/
Path created: resultsdata/swiss_climate_controversial/20200131/


In [41]:
username_list

['francisrichard',
 'MazdaArtaxerxes',
 'templivs',
 'prontipronto',
 'Chabadalala',
 'cocktail2Funk',
 'HopitalC',
 'riva_vitale',
 'Remifasol57',
 'AitiDouze',
 'QAnonAustria1',
 'gotteswerk2411']

In [42]:
min_mentions = 2 # minimal number of mentions of a user to be followed
max_day_old = 7 # number max of days in the past
exploration_depth = 2 # mention of mention of mention of ... up to exploration depth

In [43]:
total_user_list = pysad.collect_tweets(username_list, tweet_data_path, python_tweets, min_mentions=min_mentions,
               max_day_old=max_day_old, exploration_depth=exploration_depth)

  0%|          | 0/12 [00:00<?, ?it/s]

Threshold set to 2 mentions.

******* Processing users at 0-hop distance *******


  0%|          | 0/43 [00:00<?, ?it/s]

users with empty tweet list or no mention: ['francisrichard']

******* Processing users at 1-hop distance *******


 77%|███████▋  | 33/43 [00:24<00:07,  1.35it/s]

Twitter API returned error 401 for user PensePetitHans.


100%|██████████| 43/43 [00:32<00:00,  1.33it/s]

users with empty tweet list or no mention: ['Etienne_Chouard', 'PensePetitHans', 'DebnamCarey']
Total number of users collected:
236 236


In [44]:
len(total_user_list)

236

# Loading the saved data into an edge table

In [45]:
import glob

edge_df = pd.DataFrame()
for filename in glob.glob(tweet_data_path + '*_mentions' +'_t' +str(min_mentions)+ '.json'):
    new_edge_df = pd.read_json(filename)
    print('{} with {} tweets.'.format(filename,len(new_edge_df)))
    edge_df = edge_df.append(new_edge_df)
edge_df.reset_index(drop=True, inplace=True)

tweetdata/swiss_climate_controversial/20200131/Todesengel1179_mentions_t2.json with 17 tweets.
tweetdata/swiss_climate_controversial/20200131/adcdaily_mentions_t2.json with 8 tweets.
tweetdata/swiss_climate_controversial/20200131/RestitutorOrien_mentions_t2.json with 40 tweets.
tweetdata/swiss_climate_controversial/20200131/AitiDouze_mentions_t2.json with 76 tweets.
tweetdata/swiss_climate_controversial/20200131/cocktail2Funk_mentions_t2.json with 50 tweets.
tweetdata/swiss_climate_controversial/20200131/lovephilosophy_mentions_t2.json with 1 tweets.
tweetdata/swiss_climate_controversial/20200131/FridayForFuture_mentions_t2.json with 13 tweets.
tweetdata/swiss_climate_controversial/20200131/QAnonAustria1_mentions_t2.json with 34 tweets.
tweetdata/swiss_climate_controversial/20200131/MoadabJ_mentions_t2.json with 39 tweets.
tweetdata/swiss_climate_controversial/20200131/EnModeMacaron_mentions_t2.json with 10 tweets.
tweetdata/swiss_climate_controversial/20200131/Conflits_FR_mentions_t2.

In [46]:
DEGREE_MIN = 2 # Minimal number of connections in the graph

G = pysad.graph_from_edgeslist(edge_df,DEGREE_MIN)
G.name = category_name
G.date = date_string

Creating the graph from the edge list
Field "hashtags" of class <class 'list'> converted to json string
Field "date" of class <class 'list'> converted to json string
Field "urls" of class <class 'list'> converted to json string
Field "text" of class <class 'list'> converted to json string
Nb of nodes: 1928
Nb of nodes after removing nodes with degree strictly smaller than 2: 231
removed 0 isolated nodes.


## Community detection and save full graph

In [47]:
G,clusters = pysad.detect_communities(G)

Communities saved on the graph as node attributes.
Nb of partitions: 10


In [15]:
# Save the graph
import networkx as nx

graphname = 'globalgraph'

graphfilename = results_data_path + graphname + '_t' + str(min_mentions) + '_md' + str(DEGREE_MIN) +'_graph.gexf'
nx.write_gexf(G,graphfilename)
#nx.write_yaml(G,graphfilename)
#nx.write_pajek(G,graphfilename)
#nx.node_link_data(G,graphfilename)
print('Wrote',graphfilename)

Wrote resultsdata/swiss_immigration/20200131/globalgraph_t2_md2_graph.gexf


## Analyze clusters

## Automatic processing of all clusters

In [48]:
cluster_info_dic = {}
for c_id in clusters:
    cgraph = clusters[c_id]
    cgraph = pysad.cluster_attributes(cgraph)
    table_dic = pysad.cluster_tables(cgraph)
    cluster_filename = results_data_path + 'cluster' + str(c_id)
    cluster_info_dic[c_id] = {}
    cluster_info_dic[c_id]['info_table'] = table_dic
    cluster_info_dic[c_id]['filename'] = cluster_filename    

In [49]:
# Adding global infos
# keywords
corpus = pysad.get_corpus(cluster_info_dic)
keyword_dic = pysad.tfidf(corpus)

In [50]:
# gathering global info
# Saving in files
for c_id in cluster_info_dic:
    info_table = cluster_info_dic[c_id]['info_table']
    info_table['keywords'] = keyword_dic[c_id]
    pysad.save_excel(info_table,cluster_info_dic[c_id]['filename'] + '_infos.xlsx')
    pysad.save_graph(clusters[c_id],cluster_info_dic[c_id]['filename'] + 'graph.gexf')

Data saved to resultsdata/swiss_climate_controversial/20200131/cluster0_infos.xlsx
Graph saved to resultsdata/swiss_climate_controversial/20200131/cluster0graph.gexf
Data saved to resultsdata/swiss_climate_controversial/20200131/cluster1_infos.xlsx
Graph saved to resultsdata/swiss_climate_controversial/20200131/cluster1graph.gexf
Data saved to resultsdata/swiss_climate_controversial/20200131/cluster2_infos.xlsx
Graph saved to resultsdata/swiss_climate_controversial/20200131/cluster2graph.gexf
Data saved to resultsdata/swiss_climate_controversial/20200131/cluster3_infos.xlsx
Graph saved to resultsdata/swiss_climate_controversial/20200131/cluster3graph.gexf
Data saved to resultsdata/swiss_climate_controversial/20200131/cluster4_infos.xlsx
Graph saved to resultsdata/swiss_climate_controversial/20200131/cluster4graph.gexf
Data saved to resultsdata/swiss_climate_controversial/20200131/cluster5_infos.xlsx
Graph saved to resultsdata/swiss_climate_controversial/20200131/cluster5graph.gexf
Data